In [2]:
# importing required libraries for recommendation system
import pandas as pd
import numpy as np
from math import sqrt
from sklearn.decomposition import TruncatedSVD  # a dimensionality reduction technique (Singular Value Decomposition)
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
# Loading required dataset for the recommendation
ratings=pd.read_csv('ratings.csv')
ratings.head()
movies=pd.read_csv('movies.csv')
movies.head()
# creating a user item matrix
user_item_matrix=ratings.pivot_table(index='userId',columns='movieId',values='rating').fillna(0)
# Apply matrix factorization using SVD
svd=TruncatedSVD(n_components=20)
reduced_mat=svd.fit_transform(user_item_matrix)
# computing cosine similarity between users
user_similarity = cosine_similarity(reduced_mat)
# converting this similarity into dataframe
user_similarity_df=pd.DataFrame(user_similarity,index=user_item_matrix.index,columns=user_item_matrix.index)
# Function to get top n similar users
def get_similar_users(user_id,n=5):
  return user_similarity_df[user_id].sort_values(ascending=False)[1:n+1]
# Function to recommend movies
def recommend_movies(user_id,n_recommendations):
  similar_users=get_similar_users(user_id)
  similar_user_ratings=user_item_matrix.loc[similar_users.index]
  mean_ratings=similar_user_ratings.mean().sort_values(ascending=False)
  movies_seen = user_item_matrix.loc[user_id][user_item_matrix.loc[user_id] > 0].index
  recommendations = mean_ratings.drop(movies_seen).head(n_recommendations)
  return movies[movies['movieId'].isin(recommendations.index)]
def predict_ratings(user_id):
  similar_users=get_similar_users(user_id)
  similar_user_ratings=user_item_matrix.loc[similar_users.index]
  predicted_ratings=similar_user_ratings.mean()
  return predicted_ratings

actual_ratings = []
predicted_ratings = []

# Iterate over all users
for user_id in user_item_matrix.index:
    user_rated_movies = user_item_matrix.loc[user_id]
    predicted = predict_ratings(user_id)
    # Align only on movies that the user has rated
    common_movies = user_rated_movies[user_rated_movies > 0].index
    if len(common_movies) == 0:
        continue
    actual = user_rated_movies[common_movies]
    predicted_vals = predicted[common_movies]
    actual_ratings.extend(actual.values)
    predicted_ratings.extend(predicted_vals.values)
user_id=int(input("Enter the user id number: "))
n_recommendations=int(input("Enter the number of recommendations to be return for the user: "))
recommended=recommend_movies(user_id,n_recommendations)
print(f"Recommended Movies for the user with ID {user_id}")
print(recommended[['title']])
rmse = sqrt(mean_squared_error(actual_ratings, predicted_ratings))
print(f"RMSE of the recommendation system: {rmse:.4f}")

Enter the user id number:  50
Enter the number of recommendations to be return for the user:  2


Recommended Movies for the user with ID 50
                          title
46   Usual Suspects, The (1995)
461     Schindler's List (1993)
RMSE of the recommendation system: 2.8602
